In [2]:
%pip install -q kaggle
%pip install -q kagglehub
%pip install -q dask
%pip install -q ipywidgets --upgrade

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import json
import os

# Définis le chemin du fichier kaggle.json
path = r"C:\Users\f.gionnane\Documents\Data Engineering\Mes-Projets\Credentials\kaggle.json"

# Charge les credentials depuis le fichier JSON
with open(path, 'r') as file:
    content = json.load(file)

# Récupère les valeurs du fichier
username = content["username"]
api_key = content["api_key"]  # ⚠️ Assure-toi que la clé s'appelle bien "key" dans ton fichier JSON

# ✅ Définis les variables d'environnement avant de les utiliser
os.environ['KAGGLE_USERNAME'] = username
os.environ['KAGGLE_KEY'] = api_key

from kaggle.api.kaggle_api_extended import KaggleApi

# 1. **Authentification avec Kaggle**
def authenticate_kaggle():
    # Vérifie si le fichier kaggle.json est dans le bon dossier
    kaggle_json_path = os.path.expanduser('~/.kaggle/kaggle.json')  # pour Linux/macOS
    # Pour Windows, utilisez : os.path.expanduser(r'C:\Users\<Votre-Nom>\.kaggle\kaggle.json')

    # Si le fichier n'existe pas, avertir l'utilisateur
    if not os.path.exists(kaggle_json_path):
        print("Le fichier 'kaggle.json' n'est pas trouvé. Téléchargez-le depuis votre compte Kaggle.")
        print("Assurez-vous de le placer dans le répertoire ~/.kaggle/ (Linux/macOS) ou C:\\Users\\<Your-Username>\\.kaggle\\ (Windows).")
        return False  # L'authentification échoue
    else:
        print(f"Authentification réussie avec le fichier '{kaggle_json_path}'")
        api = KaggleApi()
        api.authenticate()  # Authentification avec le fichier 'kaggle.json'
        return True

# Exécuter l'authentification
authenticate_kaggle()


KeyError: 'KAGGLE_USERNAME'

In [6]:
import pandas as pd, requests
import dask as dd, kaggle
from kaggle import api # import the already authenticated API client
import kagglehub, re
import requests, subprocess
import os, sys, time
from tqdm import tqdm

OSError: Could not find kaggle.json. Make sure it's located in C:\Users\f.gionnane\.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/

In [8]:
path = r"C:\Users\f.gionnane\Downloads\Mes-Projets\Kaggle Data Projects\data"

Functions For Datasets

In [ ]:
def list_datasets(keyword=None, usability_min=None, min_mb=None, columns=None):
    print(f"Lister les datasets correspondant à '{keyword if keyword else 'tous les datasets'}' :")
    
    api = KaggleApi()
    api.authenticate()

    datasets = api.datasets_list(search=keyword) if keyword else api.datasets_list()
    
    result = []
    for dataset in datasets:
        dataset_dict = dataset  

        if columns is not None:
            dataset_dict = {key: dataset[key] for key in dataset if key in columns}

        result.append(dataset_dict)
    
    df = pd.DataFrame(result)
    df = df.loc[:, ~df.columns.str.contains('has|Nullable', case=False)]

    if "tags" in df.columns:
        df["tags"] = df["tags"].apply(
            lambda x: [d["nameNullable"] for d in x if isinstance(d, dict) and "nameNullable" in d] if isinstance(x, list) else []
        )

    if "usabilityRating" in df.columns:
        df["usabilityRating"] = round(df["usabilityRating"], 2)

    if usability_min is not None and "usabilityRating" in df.columns:
        df = df[df["usabilityRating"] >= usability_min]

    if min_mb is not None and "Size (Mb)" in df.columns:
        df = df[df["Size (Mb)"] >= min_mb]

    if keyword:
        df = df[
            df["title"].str.contains(keyword, case=False, na=False) |  
            df["tags"].apply(lambda tags: any(keyword in tag.lower() for tag in tags))
        ]

    if "url" in df.columns:
        for idx, row in df.iterrows():
            try:
                response = requests.get(row["url"])
                if response.status_code >= 400:
                    df.drop(idx, inplace=True)
            except requests.exceptions.RequestException:
                df.drop(idx, inplace=True)

    for idx, row in df.iterrows():
        ref = row["ref"]
        title = row["title"]
        
        response = requests.get(f"https://www.kaggle.com/api/v1/datasets/metadata/{ref}")
        
        if response.status_code == 200:
            content = response.json()  
            
            if content["info"]["isPrivate"]:
                df.drop(labels=idx, inplace=True)  
        
        else:
            print(f"Error fetching metadata for {title} (Status code: {response.status_code})")

    if columns is not None:
        df = df[[col for col in columns if col in df.columns]]  # Réorganiser les colonnes dans l'ordre de la liste fournie

    if "totalBytes" in df.columns:
        df["totalBytes"] = round(df["totalBytes"] / 1_000_000, 2)
        df.rename(columns={"totalBytes": "Size (Mb)"}, inplace=True)
    
    if "usabilityRating" in df.columns:
        df.rename(columns={"usabilityRating": "Usability"}, inplace=True)
    

    print(f'{df.shape[0]} résultat(s) trouvé(s) ! ')
    return df


def download_dataset(dataset_ref: str, path: str =path):
    """
    Télécharge un dataset Kaggle via l'API en créant un dossier pour chaque dataset, avec une barre de progression.

    Args:
        dataset_ref (str): Référence du dataset sous la forme "username/dataset-name".
        path (str, optionnel): Répertoire où enregistrer le fichier téléchargé. Par défaut: ".".

    Returns:
        None
    """
    dataset_identifier = dataset_ref
    # Création du dossier pour le dataset avec son nom
    dataset_folder = os.path.join(path, dataset_identifier.replace("/", "_"))

    # Si le dossier existe déjà, informer l'utilisateur
    if os.path.exists(dataset_folder):
        print(f"✅ Le dataset '{dataset_identifier}' est déjà téléchargé dans '{dataset_folder}'.")
        return

    # Créer un dossier pour le dataset
    os.makedirs(dataset_folder)
    print(f"📂 Création du dossier pour le dataset '{dataset_identifier}' dans '{dataset_folder}'...")

    # Récupérer la taille du dataset via l'API Kaggle
    metadata_url = f"https://www.kaggle.com/api/v1/datasets/metadata/{dataset_identifier}"
    try:
        response = requests.get(metadata_url)
        if response.status_code == 200:
            metadata = response.json()
            dataset_size = metadata["Size (Mb)"] / 1_000_000  # Conversion en Mo
            print(f"📥 Téléchargement de '{dataset_identifier}' ({dataset_size:.2f} MB)...")
        else:
            print("⚠️ Impossible de récupérer la taille du dataset.")
            dataset_size = None
    except requests.exceptions.RequestException:
        print("⚠️ Erreur lors de la récupération des métadonnées.")
        dataset_size = None

    # Commande de téléchargement
    command = ["kaggle", "datasets", "download", "-d", dataset_identifier, "-p", dataset_folder, "--unzip"]

    try:
        # Démarrer le téléchargement avec suivi de progression
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Barre de progression
        with tqdm(total=dataset_size, unit="MB", unit_scale=True, desc="📥 Progression") as pbar:
            while process.poll() is None:
                time.sleep(1)
                pbar.update(1)

        # Vérifier si le téléchargement a réussi
        if process.returncode == 0:
            print(f"✅ Dataset '{dataset_identifier}' téléchargé avec succès dans '{dataset_folder}'.")
        else:
            print(f"❌ Erreur lors du téléchargement. Code de retour: {process.returncode}")

    except subprocess.CalledProcessError as e:
        print(f"❌ Erreur dans la commande Kaggle. Code de retour: {e.returncode}")
        print(f"Erreur : {e.stderr.decode('utf-8')}")
    except FileNotFoundError:
        print("❌ Erreur : La commande 'kaggle' n'a pas été trouvée. Installez l'API Kaggle.")
    except Exception as e:
        print(f"❌ Une erreur inattendue s'est produite : {str(e)}")

In [19]:
columns= ['id',  'title', 'ref', 'totalBytes', 'usabilityRating', 
          'description', 'url', 'lastUpdated', 'topicCount', 
          'voteCount', 'currentVersionNumber', 'tags',
          'isPrivate']

In [20]:
df_ocean_datasets =list_datasets(keyword='finance', usability_min=0.6, min_mb=5,columns=columns)
df_ocean_datasets.head()

Lister les datasets correspondant à 'finance' :
15 résultat(s) trouvé(s) ! 


,id,title,ref,Size (Mb),usabilityRating,description,url,lastUpdated,topicCount,voteCount,currentVersionNumber,tags,isPrivate
2,55721,US Funds dataset from Yahoo Finance,stefanoleone992/mutual-funds-and-etfs,370.50,1.00,,https://www.kaggle.com/datasets/stefanoleone99...,2021-12-11T16:06:22.453Z,0,253,4,"[united states, business, finance, banking, ec...",False
3,1043241,Personal Finance,bukolafatunde/personal-finance,0.01,0.65,,https://www.kaggle.com/datasets/bukolafatunde/...,2020-12-17T15:39:22.717Z,0,70,4,"[business, finance]",False
4,877973,Finance & Accounting Courses - Udemy (13K+ cou...,jilkothari/finance-accounting-courses-udemy-13...,1.02,1.00,,https://www.kaggle.com/datasets/jilkothari/fin...,2020-09-17T12:46:12.477Z,0,180,1,"[education, beginner, exploratory data analysi...",False
5,2230,Finance ₹ - India,rajanand/finance-india,0.03,0.82,,https://www.kaggle.com/datasets/rajanand/finan...,2017-08-27T12:17:02.98Z,0,76,1,"[india, finance, lending]",False
6,3378270,Finance Loan approval Prediction Data,krishnaraj30/finance-loan-approval-prediction-...,0.01,1.00,,https://www.kaggle.com/datasets/krishnaraj30/f...,2023-06-08T16:29:50.153Z,0,85,1,"[exploratory data analysis, data visualization...",False


In [15]:
print(df_ocean_datasets.columns)
df_ocean_datasets.shape

Index(['id', 'ref', 'Size (Mb)', 'url', 'lastUpdated', 'isPrivate',
       'description', 'title', 'topicCount', 'voteCount',
       'currentVersionNumber', 'usabilityRating', 'tags'],
      dtype='object')


(5, 13)

In [15]:
for idx, row in df_ocean_datasets.iterrows():

    ref = row["ref"]
    download_dataset(dataset_ref=ref, path=path)

📂 Création du dossier pour le dataset 'stefanoleone992/mutual-funds-and-etfs' dans 'C:\Users\f.gionnane\Downloads\Mes-Projets\Kaggle Data Projects\data\stefanoleone992_mutual-funds-and-etfs'...


KeyError: 'totalBytes'

In [12]:
url_list = []
metadata_list = []
count = 0

for idx, row in df_ocean_datasets.iterrows():
    url = row["url"]
    url_list.append(url)
    ref = row["ref"]
    
    response = requests.get(f"https://www.kaggle.com/api/v1/datasets/metadata/{ref}")
    
    if response.status_code == 200:
        content = response.json()  # Stocker le JSON
        metadata_list.append(content)  # Ajouter à la liste des métadonnées
        
        # Vérifier si le dataset est privé
        if content["info"]["isPrivate"]:
            df_ocean_datasets.drop(labels=idx, inplace=True)  # Supprimer la ligne
            count += 1
            
    else:
        print(f"Error fetching metadata for {url} (Status code: {response.status_code})")

df_metadata = pd.DataFrame(metadata_list)
print(f"Nombre de datasets privés supprimés : {count}")
df_metadata.head()

Nombre de datasets privés supprimés : 0


,errorMessageNullable,info,errorMessage,hasErrorMessage
0,None,{'datasetSlugNullable': 'tiles-of-cancer-2048p...,,False
1,None,{'datasetSlugNullable': 'ubc-ocean-tiles-w-mas...,,False
2,None,{'datasetSlugNullable': 'ocean-data-climate-ch...,,False
3,None,{'datasetSlugNullable': 'big-five-personality-...,,False
4,None,{'datasetSlugNullable': 'east-sea-ocean-curren...,,False


In [13]:
# Normaliser la colonne "info" avant d'appliquer le filtrage des colonnes
df_normalized = df_metadata.join(pd.json_normalize(df_metadata["info"]))
df_normalized.drop(columns=["info"], inplace=True)

# Appliquer le filtrage des colonnes après la normalisation
df_normalized = df_normalized.loc[:, ~df_normalized.columns.str.contains('has|nullable', case=False)]

df_normalized.head()

,errorMessage,datasetId,datasetSlug,ownerUser,usabilityRating,totalViews,totalVotes,totalDownloads,title,subtitle,description,isPrivate,keywords,licenses,collaborators,data
0,,3889865,tiles-of-cancer-2048px-scale-0-25,jirkaborovec,0.875000,4926,63,980,UBC-OCEAN: Tiles🖽 of Cancer🔬 2048px|scale*0.25,Mosaic decomposition of UBC-OCEAN with scale 0...,This is a pre-processed version of the dataset...,False,"[healthcare, earth and nature, medicine, data ...",[{'nameNullable': 'Attribution-NonCommercial-N...,[],[]
1,,4027203,ubc-ocean-tiles-w-masks-2048px-scale-0-25,jirkaborovec,0.823529,2599,39,483,UBC-OCEAN: Tiles🖽 w/ masks🔬 2048px | scale 0.25,Mosaic decomposition of UBC-OCEAN with scale 0...,This is a pre-processed version of the dataset...,False,"[earth and nature, education, health conditions]","[{'nameNullable': 'other', 'name': 'other', 'h...",[],[]
2,,1601937,ocean-data-climate-change-nasa,brsdincer,0.875000,21612,45,2012,OCEAN DATA / CLIMATE CHANGE / NASA,NETCDF FORMAT / NEW / 2018-2021,### Context\n\n* Change Of Surface Skin Temper...,False,"[earth and nature, earth science, weather and ...","[{'nameNullable': 'DbCL-1.0', 'name': 'DbCL-1....",[],[]
3,,516764,big-five-personality-test,tunguz,1.000000,292985,856,30745,Big Five Personality Test,"1M Answers to 50 personality items, and techni...",### Background Information\n\nFrom [Wikipedia]...,False,[psychology],"[{'nameNullable': 'other', 'name': 'other', 'h...",[],[]
4,,1593425,east-sea-ocean-current,junhyeok99,0.823529,4410,17,319,East Sea Ocean Current,Ocean Current Data at East Sea (Asia),,False,"[earth science, beginner, time series analysis...","[{'nameNullable': 'other', 'name': 'other', 'h...",[],[]


In [28]:
print(df_normalized.shape)
df_normalized["isPrivate"].value_counts()

(7, 28)


isPrivate
False    7
Name: count, dtype: int64

Functions For Models

In [60]:
def list_models(keyword):
    print(f"Lister les modèles correspondant à '{keyword}' :")
    
    api = KaggleApi()
    api.authenticate()
    
    models = api.models_list(search=keyword)
    desired_columns = [
        'id', 'ref', 'title', 'description', 'url',
        'totalVotes', 'lastUpdated', 'tags', 'licenseName'
    ]
    
    result = []
    for model in models:
        model_dict = {key: model[key] for key in model if key in desired_columns}
        result.append(model_dict)
    
    df = pd.DataFrame(result)

    # Suppression des colonnes inutiles contenant "has" ou "Nullable"
    df = df.loc[:, ~df.columns.str.contains('has|Nullable', case=False)]
    df["tags"] = df["tags"].apply(lambda x: [d["name"] for d in x] if x is not None else [])
    # Vérification des liens URL pour voir si les modèles sont accessibles
    for idx, row in df.iterrows():
        url = row["url"]
        try:
            response = requests.get(url)
            if response.status_code >= 400:
                df.drop(idx, inplace=True)
        except requests.exceptions.RequestException:
            df.drop(idx, inplace=True)
    

    return df

In [111]:
url_list =[]

for idx, row in df_ocean_datasets.iterrows():
    url = row["url"]
    url_list.append(url)
    title = row["title"]
    if title == 'OCEAN DATA / CLIMATE CHANGE / NASA':
        try:
            download_dataset(url)
        except Exception as e:
            print(e)
    else :
        print('dataset not found')

dataset not found
dataset not found
Téléchargement du dataset : datasets/brsdincer depuis https://www.kaggle.com/datasets/brsdincer/ocean-data-climate-change-nasa...
Dataset URL: https://www.kaggle.com/datasets/datasets/brsdincer
❌ Erreur lors du téléchargement du dataset : (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json', 'Date': 'Mon, 10 Mar 2025 17:59:14 GMT', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Origin': '*', 'Set-Cookie': 'ka_sessionid=d5131f046cef14a0e32d75e5cc48286a; max-age=2626560; path=/, GCLB=CKXs7vrIkp33HRAD; path=/; HttpOnly', 'Vary': 'Accept-Encoding', 'X-Kaggle-MillisecondsElapsed': '6', 'X-Kaggle-RequestId': 'd5e46b0dec7ad7815f1b2da85596ff53', 'X-Kaggle-ApiVersion': '1.6.17', 'X-Kaggle-HubVersion': '0.3.10', 'X-Frame-Options': 'SAMEORIGIN', 'Strict-Transport-Security': 'max-age=63072000; includeSubDomains; preload', 'Content-Security-Policy': "object-src 'none'; script-src 'nonce-uuoLSZYKD9Ymq

In [ ]:
df_plus_80 = df_ocean_datasets[df_ocean_datasets["usabilityRating"]>0.8]

In [64]:
df_plus_80.head()

,id,ref,Size (Mb),url,lastUpdated,description,title,topicCount,voteCount,usabilityRating,tags
1,3889865,jirkaborovec/tiles-of-cancer-2048px-scale-0-25,53076.24,https://www.kaggle.com/datasets/jirkaborovec/t...,2023-11-08T05:33:17.887Z,,UBC-OCEAN: Tiles🖽 of Cancer🔬 2048px|scale*0.25,0,63,0.88,"[healthcare, earth and nature, medicine, data ..."
4,2628764,sudhanshu2198/oil-spill-detection,0.08,https://www.kaggle.com/datasets/sudhanshu2198/...,2022-11-12T11:34:27.303Z,,Oil Spill Classification,0,81,0.94,"[water bodies, sampling, earth and nature, ear..."
5,1968878,anandhuh/covid-in-oceania-latest-data,0.00,https://www.kaggle.com/datasets/anandhuh/covid...,2023-01-11T10:07:17.263Z,,Covid in Oceania - Latest Data,0,52,1.00,"[beginner, exploratory data analysis, tabular,..."
7,4027203,jirkaborovec/ubc-ocean-tiles-w-masks-2048px-sc...,19058.06,https://www.kaggle.com/datasets/jirkaborovec/u...,2023-11-22T18:17:53.31Z,,UBC-OCEAN: Tiles🖽 w/ masks🔬 2048px | scale 0.25,0,39,0.82,"[earth and nature, education, health conditions]"
8,1601937,brsdincer/ocean-data-climate-change-nasa,214.96,https://www.kaggle.com/datasets/brsdincer/ocea...,2021-09-20T21:04:49.183Z,,OCEAN DATA / CLIMATE CHANGE / NASA,0,45,0.88,"[earth and nature, earth science, weather and ..."
